In [1]:
import string
import re
import pandas as pd 
import numpy as np
import pymorphy2
import tokenize_uk as tk
import gensim
from collections import defaultdict

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils import shuffle, resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

import sklearn_crfsuite

from stop_words import get_stop_words

pd.set_option('display.max_colwidth', -1)
import warnings
warnings.filterwarnings('ignore')

In [14]:
df = pd.read_csv('../../data/golden_data.csv')

In [15]:
df.groupby(['y']).count()

,sentence_uk
y,
0,11112
1,11058


In [16]:
text = df['sentence_uk']
y = df['y'].values
X_train, X_test, y_train, y_test = train_test_split(text, y, test_size=0.2, random_state=1234, stratify=y)

In [17]:
stop_words = get_stop_words('uk')
morph = pymorphy2.MorphAnalyzer(lang='uk')

def clean_data(text):
    reg = re.compile("""[\"#$%&*\-+/:;<=>@^`~…\\(\\)⟨⟩{}\[\|\]‒–—―«»“”‘’№]""")
    result = text.apply(lambda sent: [re.sub(reg, '', x) for x in sent])
    result = result.apply(lambda sent: [x for x in sent if x.strip()])

    result = result.apply(lambda sent: [x.lower() for x in sent])

    return result

def lemmatize(words): 
    result_words = []
    for w in words: 
        m = morph.parse(w)[0]
        if m.normal_form:
            result_words.append(m.normal_form)
    return result_words

def delete_stop_words(words): 
    result = []
    for w in words: 
        if w not in stop_words: 
            result.append(w)
    return result

In [18]:
X_train_clean = clean_data(X_train.apply(tk.tokenize_words)).apply(lemmatize)
X_test_clean = clean_data(X_test.apply(tk.tokenize_words)).apply(lemmatize)

In [19]:
accum = {
    'x': X_test, 
    'y': y_test, 
    'x_clean': X_test_clean,
}

In [20]:
vectorizer = CountVectorizer(analyzer='word', lowercase=False, tokenizer=lambda x: x, max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train_clean.apply(delete_stop_words))
X_test_vec = vectorizer.transform(X_test_clean.apply(delete_stop_words))

In [21]:
clf = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1, learning_rate='optimal', random_state=1234)
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_test_vec)

accum['bow_y_pred'] = y_pred

f1_score(y_test, y_pred)

0.8152223750573131

In [22]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.81      0.83      0.82      2222
          1       0.83      0.80      0.82      2212

avg / total       0.82      0.82      0.82      4434



In [23]:
confusion_matrix(y_test, y_pred)

array([[1850,  372],
       [ 434, 1778]])

In [24]:
d = pd.DataFrame(accum)

In [232]:
d[d['bow_y_pred'] != d['y']].to_csv('../../data/errors_bow.csv', index=False, header=True)

In [233]:
d.head()

,bow_y_pred,x,x_clean,y
24838,0,"Про це в Facebook повідомив координатор групи ""Інформаційний опір"" Дмитро Тимчук.","[про, це, в, facebook, повідомити, координатор, група, інформаційний опір, дмитро, тимчук, .]",0
4908,1,"""У тій гострій ситуації, в якій перебувають відносини Росії і України, Росії та Заходу, ви ж розумієте, що ніхто не стане звільняти Сущенка просто так.","[у, той, гострий, ситуація, ,, в, який, перебувати, відносини, росія, і, україна, ,, росія, та, захід, ,, ви, ж, розуміти, ,, що, ніхто, не, стан, звільняти, сущенко, просто, так, .]",1
60531,0,"""Одна з комерційних структур Полтавщини підписала угоду на постачання вуглеводнів з двома нафтопереробними заводами, що розташовані у Харківській і Дніпропетровській областях.","[один, з, комерційний, структура, полтавщина, підписати, угода, на, постачання, вуглеводень, з, два, нафтопереробний, завод, ,, що, розташований, у, харківський, і, дніпропетровський, область, .]",0
642,1,Народний депутат від Блоку Петра Порошенка Микола Томенко вважає популізмом демонстративну відмову парламенту від літніх канікул.,"[народний, депутат, від, блок, петро, порошенко, микола, томенко, вважати, популізм, демонстративний, відмова, парламент, від, літній, канікули, .]",1
29775,1,"""Сьогодні всі, хто критикує уряд Яценюка, не можуть дати відповідь, а що вони пропонують.","[сьогодні, весь, ,, хто, критикувати, уряд, яценюк, ,, не, могти, дати, відповідь, ,, а, що, вони, пропонувати, .]",0
